**1532. The Most Recent Three Orders**<br>
Write an SQL query to find the most recent 3 orders of each user. If a user ordered less than 3 orders return all of their orders.

Return the result table sorted by customer_name in ascending order and in case of a tie by the customer_id in ascending order. If there still a tie, order them by the order_date in descending order.

In [ ]:
SELECT t.name AS customer_name, t.customer_id, t.order_id, t.order_date
FROM (SELECT c.name, 
             c.customer_id, 
             o.order_id, 
             o.order_date, 
             ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY o.order_date DESC) AS rown
             FROM Customers c
             JOIN Orders o
             ON c.customer_id = o.customer_id) t
WHERE t.rown<=3
ORDER BY customer_name, customer_id, order_date DESC

**1549. The Most Recent Orders for Each Product**<br>
Write a SQL query to find the most recent order(s) of each product.<br>

Return the result table sorted by product_name in ascending order and in case of a tie by the product_id in ascending order.<br> If there still a tie, order them by the order_id in ascending order.

In [ ]:
# Method 1:

SELECT p.product_name, o.product_id, o.order_id, o.order_date
FROM Orders o 
JOIN Products p
ON o.product_id = p.product_id
WHERE (o.order_date, o.product_id) 
IN (SELECT MAX(order_date) AS order_date, product_id FROM Orders GROUP BY product_id)
ORDER BY p.product_name, o.product_id, o.order_id

In [ ]:
# Method 2:

SELECT t.product_name, t.product_id, t.order_id, t.order_date
FROM (SELECT p.product_name, 
             o.product_id, 
             o.order_id, 
             o.order_date, 
             RANK() OVER (PARTITION BY o.product_id ORDER BY o.order_date DESC) AS rnk
      FROM Products p JOIN Orders o
      ON o.product_id = p.product_id) t
WHERE t.rnk = 1
ORDER BY t.product_name, t.product_id, t.order_id

**1555. Bank Account Summary**<br>
Write an SQL query to report user_id, user_name, credit (current balance after performing transactions),<br>
credit_limit_breached (check credit_limit ("Yes" or "No"))

In [ ]:
SELECT u.user_id, u.user_name, 
       IFNULL((SUM(t.amount) + u.credit),u.credit) AS credit,
       (CASE WHEN IFNULL((SUM(t.amount) + u.credit),u.credit)<0 THEN 'Yes' ELSE 'No' END) AS credit_limit_breached
FROM
    (SELECT paid_by AS user_id, SUM((-1)*amount) AS amount FROM transaction GROUP BY user_id
    UNION
    SELECT paid_to AS user_id, SUM(amount) FROM transaction GROUP BY user_id) t
RIGHT JOIN Users u 
ON u.user_id = t.user_id
GROUP BY u.user_id